In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import onnx
import onnxruntime as ort
import os
from scipy.io import savemat, loadmat
from tqdm import tqdm
import torch
from pprint import pprint

In [ ]:
model_path = './ckpt/combined_model.onnx'

onnx_model = onnx.load(model_path)
onnx.checker.check_model(onnx_model)

input_names = [input.name for input in onnx_model.graph.input]
output_names = [output.name for output in onnx_model.graph.output]
print(f'Model inputs: {input_names}')
print(f'Model output: {output_names}')

del onnx_model

In [ ]:
def save_results(
        results_dir,
        sample_idx,
        image,
        gt_edges,
        gt_corners,
        pred_edges,
        pred_corners,
        room_type
):
    image = Image.fromarray((image * 255).astype(np.uint8).transpose(1, 2, 0))
    
    gt_edges_image = Image.fromarray((gt_edges * 255).astype(np.uint8).transpose(1, 2, 0))
    gt_corners_image = Image.fromarray(
        (gt_corners.sum(axis=0) * 255).astype(np.uint8))
    
    pred_edges_image = Image.fromarray((pred_edges * 255).astype(np.uint8).transpose(1, 2, 0))
    pred_corners_image = Image.fromarray(
        (pred_corners.sum(axis=0) * 255).astype(np.uint8))
    
    base_file = os.path.join(results_dir, f'{sample_idx}')
    image.save(f'{base_file}_rt-{room_type}.jpg')
    gt_edges_image.save(f'{base_file}_gt_edges.jpg')
    gt_corners_image.save(f'{base_file}_gt_corners.jpg')
    pred_edges_image.save(f'{base_file}_pred_edges.jpg')
    pred_corners_image.save(f'{base_file}_pred_corners.jpg')

    np.save(f'{base_file}_pred_corners.npy', pred_corners)

In [ ]:
def save_results_mat(
    results_dir,
    sample_idx,
    image,
    pred_edges,
    pred_corners,
    pred_types
):
    ...

Run inference on test set and save results in the same format as original Lua Torch LayoutNet project

In [ ]:
REINFER = False

src_images_dir = '/home/richard/Development/LayoutNet/data/lsun_val/img'
results_dir = 'res/lsun_val'
model_path = './ckpt/combined_model.onnx'

ort_sess = ort.InferenceSession(model_path)

cor_dir = os.path.join(results_dir, 'cor')
cor_mat_dir = os.path.join(results_dir, 'cor_mat')
cor_mat_flip_dir = os.path.join(results_dir, 'cor_mat_flip')
edg_dir = os.path.join(results_dir, 'edg')
img_dir = os.path.join(results_dir, 'img')
type_dir = os.path.join(results_dir, 'type')
type_gt_dir = os.path.join(results_dir, 'type_gt')

for p in [cor_dir, cor_mat_dir, cor_mat_flip_dir, edg_dir, img_dir, type_dir]:
    os.makedirs(p, exist_ok=True)

def is_image(fname):
    ext = fname.split('.')[-1]
    return ext in {'jpg', 'jpeg', 'png'}

src_images = sorted(filter(is_image, os.listdir(src_images_dir)))

if REINFER:

    for idx, image_file_name in tqdm(enumerate(src_images), total=len(src_images)):   
        image_name = image_file_name.split('.')[0]
        
        image = Image.open(os.path.join(src_images_dir, image_file_name))
        image = image.resize((512, 512))
        image = np.array(image)
        image = image.astype(np.float32) / 255.0

        image_flip = np.fliplr(image).copy()

        image = image.transpose(2, 0, 1)
        image_flip = image_flip.transpose(2, 0, 1)

        x = np.expand_dims(image, axis=0)
        x_flip = np.expand_dims(image_flip, axis=0)
        
        outputs = ort_sess.run(
            None,
            {
                'input': x
            }
        )
        outputs_flip = ort_sess.run(
            None,
            {
                'input': x_flip
            }
        )

        edges_out, corners_out, room_type_out = outputs
        edges_out_flip, corners_out_flip, room_type_out_flip = outputs_flip

        room_type_arr = np.stack([room_type_out[0], room_type_out_flip[0]], axis=0)
        room_type_mat = {
            'x': room_type_arr
        }

        edges_out_unflip =  np.fliplr(edges_out_flip[0].transpose(1, 2, 0)).transpose(2, 0, 1)
        edges_arr = np.stack([edges_out[0], edges_out_unflip]).mean(axis=0)
        edges_img = (edges_arr.transpose(1, 2, 0) * 255).astype(np.uint8)
        edges_img = Image.fromarray(edges_img)

        cor_mat = {
            'x': corners_out[0]
        }
        cor_mat_flip = {
            'x': corners_out_flip[0]
        }

        cor_unflipped_sum = np.fliplr(corners_out_flip[0].sum(axis=0)).clip(0, 1)
        cor_sum = corners_out[0].sum(axis=0).clip(0, 1)

        cor_img = np.stack([
            cor_unflipped_sum,  
            cor_sum
        ]).mean(axis=0)
        cor_img = (cor_img * 255).astype(np.uint8)
        # Our images dont actually look like in Torch7 inference, they are flipped over there.
        cor_img = Image.fromarray(cor_img)

        img = (image.transpose(1, 2, 0) * 255).astype(np.uint8)
        img = Image.fromarray(img)

        base_file_name = str(idx + 1)
        # TODO: create room_type_gt_mat in the same form as room_type_mat from the single 'id' value
        savemat(os.path.join(type_dir, f'{base_file_name}.mat'), room_type_mat)
        edges_img.save(os.path.join(edg_dir, f'{base_file_name}.png'))
        savemat(os.path.join(cor_mat_dir, f'{base_file_name}.mat'), cor_mat)
        savemat(os.path.join(cor_mat_flip_dir, f'{base_file_name}.mat'), cor_mat_flip)
        cor_img.save(os.path.join(cor_dir, f'{base_file_name}.png'))
        img.save(os.path.join(img_dir, f'{base_file_name}.png')) 

del ort_sess

## Reformat Ground Truth Data into Prediction Format

In [ ]:
src_gt_dir = '/home/richard/Development/LayoutNet/data/lsun_val'
src_gt_img_dir = os.path.join(src_gt_dir, 'img')
src_gt_cor_dir = os.path.join(src_gt_dir, 'cor')
src_gt_cor_flip_dir = os.path.join(src_gt_dir, 'cor_flip')
src_gt_edg_dir = os.path.join(src_gt_dir, 'edg')
src_gt_type_dir = os.path.join(src_gt_dir, 'type')

dst_gt_dir = './res/lsun_val_gt'
dst_gt_img_dir = os.path.join(dst_gt_dir, 'img')
dst_gt_cor_dir = os.path.join(dst_gt_dir, 'cor')
dst_gt_cor_mat_dir = os.path.join(dst_gt_dir, 'cor_mat')
dst_gt_cor_mat_flip_dir = os.path.join(dst_gt_dir, 'cor_mat_flip')
dst_gt_edg_dir = os.path.join(dst_gt_dir, 'edg')
dst_gt_type_dir = os.path.join(dst_gt_dir, 'type')

for dir in [dst_gt_img_dir, dst_gt_cor_dir, dst_gt_cor_mat_dir, dst_gt_cor_mat_flip_dir, dst_gt_edg_dir, dst_gt_type_dir]:
    os.makedirs(dir, exist_ok=True)

In [ ]:
gt_image_file_names = list(sorted(filter(is_image, os.listdir(src_gt_img_dir))))
gt_image_file_names[0]

In [ ]:
REFORMAT = False

if REFORMAT:
    for idx, gt_image_file_name in tqdm(enumerate(gt_image_file_names), total=len(gt_image_file_names)):
        gt_sample_name = gt_image_file_name.replace('.png', '')
        gt_mat_name = f'{gt_sample_name}.mat'
        
        gt_img = Image.open(os.path.join(src_gt_img_dir, gt_image_file_name))
        gt_edg = Image.open(os.path.join(src_gt_edg_dir, gt_image_file_name))
        gt_cor = loadmat(os.path.join(src_gt_cor_dir, gt_mat_name))['cor']
        gt_cor_flip = loadmat(os.path.join(src_gt_cor_flip_dir, gt_mat_name))['cor_f']
        gt_type_id = loadmat(os.path.join(src_gt_type_dir, gt_mat_name))['id']
        
        # Transform
        gt_type_id = int(gt_type_id.squeeze())
        gt_type = np.zeros((2, 11))
        gt_type[:, gt_type_id - 1] = 1
        
        # HWC -> CHW
        gt_cor = np.transpose(gt_cor, (2, 0, 1))
        gt_cor_flip = np.transpose(gt_cor_flip, (2, 0, 1))
        
        gt_cor_unflipped_sum = np.fliplr(gt_cor.sum(axis=0)).clip(0, 1)
        gt_cor_sum = gt_cor.sum(axis=0).clip(0, 1)

        gt_cor_img = np.stack([
            gt_cor_unflipped_sum,  
            gt_cor_sum
        ]).mean(axis=0)
        gt_cor_img = (gt_cor_img * 255).astype(np.uint8)
        gt_cor_img = Image.fromarray(gt_cor_img)
        
        # TODO: create mats
        gt_room_type_mat = {
            'x': gt_type
        }
        gt_cor_mat = {
            'x': gt_cor
        }
        gt_cor_mat_flip = {
            'x': gt_cor_flip
        }
        
        base_file_name = str(idx + 1)
        # TODO: create room_type_gt_mat in the same form as room_type_mat from the single 'id' value
        savemat(os.path.join(dst_gt_type_dir, f'{base_file_name}.mat'), gt_room_type_mat)
        gt_edg.save(os.path.join(dst_gt_edg_dir, f'{base_file_name}.png'))
        savemat(os.path.join(dst_gt_cor_mat_dir, f'{base_file_name}.mat'), gt_cor_mat)
        savemat(os.path.join(dst_gt_cor_mat_flip_dir, f'{base_file_name}.mat'), gt_cor_mat_flip)
        gt_cor_img.save(os.path.join(dst_gt_cor_dir, f'{base_file_name}.png'))
        gt_img.save(os.path.join(dst_gt_img_dir, f'{base_file_name}.png'))       

## Compare Predictions with Ground-Truth

In [ ]:
gt_dir = './res/lsun_val_gt'
pred_dir = './res/lsun_val'

pred_type_dir = os.path.join(pred_dir, 'type')
gt_type_dir = os.path.join(gt_dir, 'type')

num_samples = len(os.listdir(pred_type_dir))
print(f'Number of samples: {num_samples}')

In [ ]:
gt_to_wrong_preds = {}

num_correct = 0
for idx in range(1, num_samples + 1):
    mat_file_name = f'{idx}.mat'

    gt_type = loadmat(os.path.join(gt_type_dir, mat_file_name))['x'].mean(axis=0).argmax()
    pred_type = loadmat(os.path.join(pred_type_dir, mat_file_name))['x'].mean(axis=0).argmax()
    if gt_type != pred_type:
        if gt_type not in gt_to_wrong_preds:
            gt_to_wrong_preds[gt_type] = []
        gt_to_wrong_preds[gt_type].append((idx, pred_type))
    else:
        num_correct += 1

print(f'Number of correct type predictions: {num_correct}')
print(list(sorted(gt_to_wrong_preds.keys())))

In [ ]:
for gt_type, wrong_entries in sorted(gt_to_wrong_preds.items()):
    print(f'{gt_type}: {len(wrong_entries)}')

In [ ]:
gt_to_wrong_preds[7]